# LangChain: RAG Document Example

In [1]:
# 下載範例檔案
!wget https://github.com/taipingeric/MIP/releases/download/langchain/example_data.zip
!unzip example_data.zip

--2025-11-08 07:17:41--  https://github.com/taipingeric/MIP/releases/download/langchain/example_data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/678249561/7ee5ae67-19d5-430f-932d-a2557651d547?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-08T07%3A58%3A31Z&rscd=attachment%3B+filename%3Dexample_data.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-08T06%3A58%3A03Z&ske=2025-11-08T07%3A58%3A31Z&sks=b&skv=2018-11-09&sig=AftNQ8rHIl4hFCyNftM61k5x7RF4M%2B33OGGX4JakEY4%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MjU4NjU2MSwibmJmIjoxNzYyNTg2MjYxLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5

### Install package

In [2]:
!pip install -q --upgrade langchain langchain-community langchain-openai langchain-core chromadb faiss-cpu pypdf
# 安裝langchain + google or openai
!pip install -q "langchain[google-genai]" "langchain[openai]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.2/471.2 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.8 MB/s eta 0:00:

### 設定

In [3]:
from langchain.chat_models import init_chat_model
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma, FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_classic.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from rich.pretty import pprint
from google.colab import userdata
import os

# 設定API Key
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
# 建立模型
model = init_chat_model("google_genai:gemini-2.5-flash-lite")
OPENAI_KEY = userdata.get("OPENAI_API_KEY")

## 範例: RAG: PDF Retrieval

In [4]:
# 1. 載入 PDF 文件
loader = PyPDFLoader("/content/學生個人獎懲辦法.pdf")
documents = loader.load()
pprint(documents)

[
│   Document(
│   │   metadata={
│   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   'author': 'joyceguo',
│   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   'total_pages': 6,
│   │   │   'page': 0,
│   │   │   'page_label': '1'
│   │   },
│   │   page_content='1  \n國立臺灣大學學生個人獎懲辦法 \n                                              72.9.29第72次訓育委員會議立法通過 \n                                      102.6.26學生輔導委員會第34次會議修正通過 \n                                      107.1.17學生輔導委員會第43次會議討論通過 \n             107.3.24本校106學年度第2學期第1次校務會議通過修正第3、7、18、19條 \n107.5.23教育部臺教學(二)字第1070072716號函備查 \n109.3.21本校108學年度第2學期第1次校務會議通過修正第8、9條 \n110.3.20本校109學年度第2學期第1次校務會議通過修正第3、7、8、9、10條 \n110.4.22教育部臺教學(二)字第1100051150號函備查 \n第一條 為樹立本校學生敦品勵學、愛國愛人之優良校風，確收教育功效，依\n據大學法第三十二條規定訂定本辦法。 \n第二條 本校學生個人之獎懲依本辦法辦理之。 \n第三條 學生個人之獎勵方式如下： \n一、嘉獎。 \n二、小功。 \n三、大功。 \n學生個人之懲處方式如下： \n一、書面告誡。 \n二、申誡。 \n三、小過。 \n四、大過。 \n五、勒令退學。\n六、開除學籍。 \n為前項懲處時，得建議行為人接受心理輔導或轉介身心科門診之附帶\n決議。 \n學生個人之獎懲記錄，記嘉獎三次視同記小功一次；記小功三次視同\n記大功一次；記申誡三次視同記小過一次；記小過三次視同記大過一\n次。 \n第一項所列獎勵與第二項所列懲處不得相抵。但學生初犯本辦法之規\n定而未達記大過之處分者，得依「國立臺灣大學學生消過實施要點」\n消過。受懲處學生違反第三項附帶決議者，不得申請消過。第四項\n「國立臺灣大學學生消過實施要點」另訂之。 \n第四條 學生個人有下列各款情形之一者﹐予以嘉獎： \n一、服務公勤，熱心公益，有具體事蹟。'
│   ),
│   Document(
│   │   metadata={
│   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   'author': 'joyceguo',
│   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   'total_pages': 6,
│   │   │   'page': 1,
│   │   │   'page_label': '2'
│   │   },
│   │   page_content='2  \n二、擔任學生社團幹部或自治幹部，表現優良。\n三、熱心參加體育及課外活動，表現優良。 \n四、推動校內環境保護，杜絕資源浪費，保障校園環境保護暨安全衛生\n(以下簡稱環安衛)，有具體優良事蹟。 \n五、拾物不昧。 \n六、熱心助人，義行可嘉。\n七、有其他相當之情事。 \n第五條 學生個人有下列各款情形之一者，予以記小功： \n一、服務公勤，熱心公益，表現特優。 \n二、擔任學生社團幹部或自治幹部，表現特優。\n三、熱心參加體育及課外活動，表現特優。 \n四、參加全國性比賽成績在前三名。 \n五、熱心助人，表現特優。 \n六、見義勇為，有具體優良事蹟。 \n七、推動環安衛事務有具體優良事蹟，且獲校外單位表揚。 \n八、有其他相當之情事。 \n第六條 學生個人有下列各款情形之一者，予以記大功： \n一、對社會、學校有重大貢獻，事蹟卓著。 \n二、擔任學生團幹部或自治幹部，表現特優，對樹立優良校風有特殊\n貢獻。 \n三、參加國際性比賽表現特優。 \n四、協助同學解除危困，對樹立優良校風有特殊貢獻。 \n五、特殊義勇表現，事蹟卓著。 \n六、有其他相當之情事。 \n \n第七條 學生個人有下列各款情形之一者，予以申誡： \n一、擔任學生社團幹部或自治幹部，怠忽職守致生損害。 \n二、毀損或妨害合法張貼之海報。 \n三、未經許可私自挪用他人物品。 \n四、在公共場所擾亂公共秩序。 \n五、污染校產、違反環安衛相關法令規定，而未釀成事件發生。 \n六、違反網路使用規範。 \n七、以跟蹤、電子郵件或其他違反他人意願之方法干擾他人日常生活\n之行為，情節輕微。'
│   ),
│   Document(
│   │   metadata={
│   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   'author': 'joyceguo',
│   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   'total_pages': 6,
│   │   │   'page': 2,
│   │   │   'page_label': '3'
│   │   },
│   │   page_content='3  \n八、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員會\n查證屬情節輕微，並移送懲處。 \n學生個人有前項各款情形，情節輕微，顯可憫恕，依第十二條之規定\n減輕其懲處者，得予以書面告誡。 \n第八條 學生個人有下列各款情形之一者，予以記小過： \n一、考試作弊，情節輕微。 \n二、侮辱、誹謗他人。 \n三、意圖干擾他人，公然為猥褻之行為。\n四、有公然賭博之行為。 \n五、冒用他人證件或將己有證件借與他人使用。 \n六、住宿生未經室友及宿舍管理人員之同意而留宿他人。\n七、私自頂讓學校宿舍床位。 \n八、毆人或與人互毆。 \n九、不法侵入學校廳舍、他人研究室、或寢室。\n十、毀損公物或他人之物。 \n十一、管理公物未盡善良管理人之注意義務，致生毀損、滅失、短少 \n，或管理公款有浮報、挪用、帳目不清之情事。 \n十二、侵犯他人智慧財產權。 \n十三、污染校產、違反環安衛相關法令規定，造成災害傷及自身或他 \n 人。 \n十四、違反網路使用規範，情節嚴重。\n十五、製造噪音，情節嚴重。 \n十六、在公共場所擾亂公共秩序，情節嚴重。\n十七、違反學術倫理，情節輕微。 \n十八、以跟蹤、電子郵件或其他違反他人意願之方法干擾他人日常生\n活之行為，情節嚴重。 \n十九、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員\n會查證屬實並移送懲處。 \n二十、親密關係暴力行為，且經法院核發有保護令或有具體事證者。 \n第九條 學生個人有下列各款情形之一者，予以記大過： \n一、考試作弊。 \n二、侮辱、誹謗他人，情節嚴重。\n三、竊盜、侵占、詐欺。 \n四、偽造、變造文書。'
│   ),
│   Document(
│   │   metadata={
│   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   'author': 'joyceguo',
│   │   │   

In [5]:
# 印出Documents內容
for doc in documents:
    print(doc.page_content)
    print("======= \n")

1  
國立臺灣大學學生個人獎懲辦法 
                                              72.9.29第72次訓育委員會議立法通過 
                                      102.6.26學生輔導委員會第34次會議修正通過 
                                      107.1.17學生輔導委員會第43次會議討論通過 
             107.3.24本校106學年度第2學期第1次校務會議通過修正第3、7、18、19條 
107.5.23教育部臺教學(二)字第1070072716號函備查 
109.3.21本校108學年度第2學期第1次校務會議通過修正第8、9條 
110.3.20本校109學年度第2學期第1次校務會議通過修正第3、7、8、9、10條 
110.4.22教育部臺教學(二)字第1100051150號函備查 
第一條 為樹立本校學生敦品勵學、愛國愛人之優良校風，確收教育功效，依
據大學法第三十二條規定訂定本辦法。 
第二條 本校學生個人之獎懲依本辦法辦理之。 
第三條 學生個人之獎勵方式如下： 
一、嘉獎。 
二、小功。 
三、大功。 
學生個人之懲處方式如下： 
一、書面告誡。 
二、申誡。 
三、小過。 
四、大過。 
五、勒令退學。
六、開除學籍。 
為前項懲處時，得建議行為人接受心理輔導或轉介身心科門診之附帶
決議。 
學生個人之獎懲記錄，記嘉獎三次視同記小功一次；記小功三次視同
記大功一次；記申誡三次視同記小過一次；記小過三次視同記大過一
次。 
第一項所列獎勵與第二項所列懲處不得相抵。但學生初犯本辦法之規
定而未達記大過之處分者，得依「國立臺灣大學學生消過實施要點」
消過。受懲處學生違反第三項附帶決議者，不得申請消過。第四項
「國立臺灣大學學生消過實施要點」另訂之。 
第四條 學生個人有下列各款情形之一者﹐予以嘉獎： 
一、服務公勤，熱心公益，有具體事蹟。

2  
二、擔任學生社團幹部或自治幹部，表現優良。
三、熱心參加體育及課外活動，表現優良。 
四、推動校內環境保護，杜絕資源浪費，保障校園環境保護暨安全衛生
(以下簡稱環安衛)，有具體優良事蹟。 
五、拾物不昧。 
六、熱心助人，義行可嘉。
七、有其他相當之情事。

In [6]:
# 3. 建立向量資料庫（知識庫）
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
# FAISS
db = FAISS.from_documents(documents, embedding)

In [7]:
# 4. 用 RetrievalQA 包一層問答機制
retriever = db.as_retriever() # 取得DB對應檢索器
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",  # 這裡也可以換成 refine、map_reduce 看需要
    return_source_documents=True  # 回傳來源
)
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     chain_type="stuff"  # 這裡也可以換成 refine、map_reduce 看需要
# )

In [8]:
# 5. 開始問問題
while True:
    query = input("請問你想問什麼？（輸入 'exit' 結束）\n")
    if query.lower() == "exit":
        break
    response = qa_chain.invoke(query)
    pprint(response)

請問你想問什麼？（輸入 'exit' 結束）
上課玩手機的懲罰


{
│   'question': '上課玩手機的懲罰',
│   'answer': 'I am sorry, but I cannot find the answer to your question in the provided text. The text outlines disciplinary actions for students for various offenses, but it does not specifically mention the punishment for playing on a mobile phone during class.\n',
│   'sources': '',
│   'source_documents': [
│   │   Document(
│   │   │   id='c4a1d378-c1a8-4e83-8c21-e3f386af6b9b',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   │   'total_pages': 6,
│   │   │   │   'page': 3,
│   │   │   │   'page_label': '4'
│   │   │   },
│   │   │   page_content='4  \n五、於校園內不法儲存危險物或違禁物。 \n六、不法持有毒品、安非他命或其他麻醉藥品。 \n七、不法侵入學校廳舍、他人研究室、或寢室，情節嚴重。 \n八、私自頂讓床位並獲取不當利益、霸佔學校宿舍床位、或排斥他人合\n法住進。 \n九、惡意侵入電腦網站破壞系統、資料或發送郵件炸彈危及電腦主機安\n全、干擾他人電磁紀錄之處理。 \n十、利用電腦網路或以其他方式販售、提供、或教唆製造不法商品。 \n十一、管理公物未盡善良管理人之注意義務，致生毀損、滅失、短少，\n或管理公款有浮報、挪用、帳目不清之情事，情節嚴重。 \n十二、毆人或與人互毆，情節嚴重。 \n十三、侵犯他人智慧財產權，情節嚴重。 \n十四、其他觸犯刑事法律之行為，經法院有罪判決確定或學校查證屬實 \n，情節輕微。 \n十五、污染校產，違反環安衛相關法令規定，情節重大者。\n十六、違反學術倫理，情節嚴重。 \n十七、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員\n會查證屬情節嚴重，並移送懲處。 \n十八、性侵害，經本校性別平等教育委員會或性騷擾防治委員會查證\n屬實並移送懲處。 \n十九、親密關係暴力行為，情節嚴重，且經法院核發有保護令或有具\n體事證者。 \n第十條 學生個人有下列各款情形之一者，予以勒令退學： \n一、毆打師長。 \n二、考試作弊，情節嚴重。 \n三、違反學術倫理，情節極為嚴重。\n四、蓄意傷人，情節嚴重。 \n五、不法販賣或製造毒品、安非他命或其他麻醉藥品。 \n六、其他觸犯刑事法律之行為，經法院有罪判決確定或學校查證屬實。 \n七、積滿三次大過。 \n八、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員 \n會查證屬情節極為嚴重，並移送懲處。 \n九、性侵害，經本校性別平等教育委員會或性騷擾防治委員會查證屬 \n情節嚴重，並移送懲處。'
│   │   ),
│   │   Document(
│   │   │   id='f8098b55-0c96-414b-a8f2-60c1fd42a1ce',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   │   'total_pages': 6,
│   │   │   │   'page': 4,
│   │   │   │   'page_label': '5'
│   │   │   },
│   │   │   page_content='5  \n第十一條  學生個人有下列各款情形之一者，予以開除學籍：\n一、毆打師長，情節嚴重。 \n二、連續考試作弊，情節嚴重。 \n三、不法販賣或製造毒品、安非他命或其他麻醉藥品，情節嚴重者。 \n四、其他觸犯刑事法律之行為，經法院有罪判決確定或學校查證屬 實 \n，情節嚴重。 \n第十二條  學生個人行為之懲戒，除依照本辦法所列標準訂定外，得酌量下列各款\n之情形為加重或減輕： \n一、行為之動機、目的。 \n二、行為時所受之刺激。 \n三、行為之手段。 \n四、行為人之生活狀況、品行。 \n五、行為人與被害人平日之關係。\n六、行為所生之危險或損害程度。\n七、行為後之態度。 \n第十三條  學生於學校處理其個人懲戒處分之過程中，湮滅證據、故意提供或唆\n使他人提供不實證據或資料者，得加重處分。 \n第十四條  學校為發現懲戒事實之真相，應委請被付懲戒學生之導師協助進行瞭\n解，並請其以書面告知。 \n第十五條  學校審議懲戒案件應通知當事學生到會陳述，並應請相關導師列席會\n議。\n第十六條  \n一、學生個人之行為接受懲戒，應由學校以書面通知本人。學生個人\n之行為記小過以上者，應由學校以書面通知家長或監護人。 \n二、學生個人之行為接受懲戒，其行為致使特定他人權益受損害者，\n經學校認為有其必要，亦得以書面轉知該特定被害人。 \n \n第十七條  前條獎懲之通知，應附記下列事項： \n           一、學生初犯本辦法之規定而未達記大過之處分者，得依本校學生消\n過實施要點，申請以愛校服務代替之。 \n           二、學生對獎懲如有不服時，得依本校學生申訴評議辦法所訂之期限\n提出申訴。 \n第十八條  學生於在學期間有勒令退學或開除學籍之懲戒事由，且於畢業或退學'
│   │   ),
│   │   Document(
│   │   │   id='126c1097-6f34-4250-a7f5-d6f8bf30c74d',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   │   'total_pages': 6,
│   │   │   │   'page': 5,
│   │   │   │   'page_label': '6'
│   │   │   },
│   │   │   page_content='6  \n後一年內被提報懲戒者，學生獎懲委員會得懲戒之。 \n          前項情形，應通知受懲戒人到場陳述意見。受懲戒人經合法通知且無\n正當理由不到場者，學生獎懲委員會得不待其陳述作成處分。受懲戒\n人未經合法通知者，應停止懲戒程序。 \n第十九條  本辦法經學生輔導委員會議通過，送校務會議核備報請教育部備查後 \n  ，自發布日施行。'
│   │   ),
│   │   Document(
│   │   │   id='6e4161c6-9db4-4e63-a277-ea94df92d104',
│   │   │   metadata={
│   │   │   │   'producer':

請問你想問什麼？（輸入 'exit' 結束）



{
│   'question': '',
│   'answer': 'The provided documents do not contain information about what the president said about Michael Jackson.\n',
│   'sources': '',
│   'source_documents': [
│   │   Document(
│   │   │   id='c4a1d378-c1a8-4e83-8c21-e3f386af6b9b',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   │   'total_pages': 6,
│   │   │   │   'page': 3,
│   │   │   │   'page_label': '4'
│   │   │   },
│   │   │   page_content='4  \n五、於校園內不法儲存危險物或違禁物。 \n六、不法持有毒品、安非他命或其他麻醉藥品。 \n七、不法侵入學校廳舍、他人研究室、或寢室，情節嚴重。 \n八、私自頂讓床位並獲取不當利益、霸佔學校宿舍床位、或排斥他人合\n法住進。 \n九、惡意侵入電腦網站破壞系統、資料或發送郵件炸彈危及電腦主機安\n全、干擾他人電磁紀錄之處理。 \n十、利用電腦網路或以其他方式販售、提供、或教唆製造不法商品。 \n十一、管理公物未盡善良管理人之注意義務，致生毀損、滅失、短少，\n或管理公款有浮報、挪用、帳目不清之情事，情節嚴重。 \n十二、毆人或與人互毆，情節嚴重。 \n十三、侵犯他人智慧財產權，情節嚴重。 \n十四、其他觸犯刑事法律之行為，經法院有罪判決確定或學校查證屬實 \n，情節輕微。 \n十五、污染校產，違反環安衛相關法令規定，情節重大者。\n十六、違反學術倫理，情節嚴重。 \n十七、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員\n會查證屬情節嚴重，並移送懲處。 \n十八、性侵害，經本校性別平等教育委員會或性騷擾防治委員會查證\n屬實並移送懲處。 \n十九、親密關係暴力行為，情節嚴重，且經法院核發有保護令或有具\n體事證者。 \n第十條 學生個人有下列各款情形之一者，予以勒令退學： \n一、毆打師長。 \n二、考試作弊，情節嚴重。 \n三、違反學術倫理，情節極為嚴重。\n四、蓄意傷人，情節嚴重。 \n五、不法販賣或製造毒品、安非他命或其他麻醉藥品。 \n六、其他觸犯刑事法律之行為，經法院有罪判決確定或學校查證屬實。 \n七、積滿三次大過。 \n八、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員 \n會查證屬情節極為嚴重，並移送懲處。 \n九、性侵害，經本校性別平等教育委員會或性騷擾防治委員會查證屬 \n情節嚴重，並移送懲處。'
│   │   ),
│   │   Document(
│   │   │   id='6e4161c6-9db4-4e63-a277-ea94df92d104',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   │   'total_pages': 6,
│   │   │   │   'page': 2,
│   │   │   │   'page_label': '3'
│   │   │   },
│   │   │   page_content='3  \n八、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員會\n查證屬情節輕微，並移送懲處。 \n學生個人有前項各款情形，情節輕微，顯可憫恕，依第十二條之規定\n減輕其懲處者，得予以書面告誡。 \n第八條 學生個人有下列各款情形之一者，予以記小過： \n一、考試作弊，情節輕微。 \n二、侮辱、誹謗他人。 \n三、意圖干擾他人，公然為猥褻之行為。\n四、有公然賭博之行為。 \n五、冒用他人證件或將己有證件借與他人使用。 \n六、住宿生未經室友及宿舍管理人員之同意而留宿他人。\n七、私自頂讓學校宿舍床位。 \n八、毆人或與人互毆。 \n九、不法侵入學校廳舍、他人研究室、或寢室。\n十、毀損公物或他人之物。 \n十一、管理公物未盡善良管理人之注意義務，致生毀損、滅失、短少 \n，或管理公款有浮報、挪用、帳目不清之情事。 \n十二、侵犯他人智慧財產權。 \n十三、污染校產、違反環安衛相關法令規定，造成災害傷及自身或他 \n 人。 \n十四、違反網路使用規範，情節嚴重。\n十五、製造噪音，情節嚴重。 \n十六、在公共場所擾亂公共秩序，情節嚴重。\n十七、違反學術倫理，情節輕微。 \n十八、以跟蹤、電子郵件或其他違反他人意願之方法干擾他人日常生\n活之行為，情節嚴重。 \n十九、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員\n會查證屬實並移送懲處。 \n二十、親密關係暴力行為，且經法院核發有保護令或有具體事證者。 \n第九條 學生個人有下列各款情形之一者，予以記大過： \n一、考試作弊。 \n二、侮辱、誹謗他人，情節嚴重。\n三、竊盜、侵占、詐欺。 \n四、偽造、變造文書。'
│   │   ),
│   │   Document(
│   │   │   id='ec4c5286-ab7e-4c00-8208-14ea8e543909',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   │   'total_pages': 6,
│   │   │   │   'page': 1,
│   │   │   │   'page_label': '2'
│   │   │   },
│   │   │   page_content='2  \n二、擔任學生社團幹部或自治幹部，表現優良。\n三、熱心參加體育及課外活動，表現優良。 \n四、推動校內環境保護，杜絕資源浪費，保障校園環境保護暨安全衛生\n(以下簡稱環安衛)，有具體優良事蹟。 \n五、拾物不昧。 \n六、熱心助人，義行可嘉。\n七、有其他相當之情事。 \n第五條 學生個人有下列各款情形之一者，予以記小功： \n一、服務公勤，熱心公益，表現特優。 \n二、擔任學生社團幹部或自治幹部，表現特優。\n三、熱心參加體育及課外活動，表現特優。 \n四、參加全國性比賽成績在前三名。 \n五、熱心助人，表現特優。 \n六、見義勇為，有具體優良事蹟。 \n七、推動環安衛事務有具體優良事蹟，且獲校外單位表揚。 \n八、有其他相當之情事。 \n第六條 學生個人有下列各款情形之一者，予以記大功： \n一、對社會、學校有重大貢獻，事蹟卓著。 \n二、擔任學生團幹部或自治幹部，表現特優，對樹立優良校風有特殊\n貢獻。 \n三、參加國際性比賽表現特優。 \n四、協助同學解除危困，對樹立優良校風有特殊貢獻。 \n五、特殊義勇表現，事蹟卓著。 \n六、有其他相當之情事。 \n \n第七條 學生個人有下列各款情形之一者，予以申誡

請問你想問什麼？（輸入 'exit' 結束）
暴打老師的懲罰


{
│   'question': '暴打老師的懲罰',
│   'answer': '毆打師長，情節嚴重者，予以開除學籍。\n',
│   'sources': '/content/學生個人獎懲辦法.pdf',
│   'source_documents': [
│   │   Document(
│   │   │   id='f8098b55-0c96-414b-a8f2-60c1fd42a1ce',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   │   'total_pages': 6,
│   │   │   │   'page': 4,
│   │   │   │   'page_label': '5'
│   │   │   },
│   │   │   page_content='5  \n第十一條  學生個人有下列各款情形之一者，予以開除學籍：\n一、毆打師長，情節嚴重。 \n二、連續考試作弊，情節嚴重。 \n三、不法販賣或製造毒品、安非他命或其他麻醉藥品，情節嚴重者。 \n四、其他觸犯刑事法律之行為，經法院有罪判決確定或學校查證屬 實 \n，情節嚴重。 \n第十二條  學生個人行為之懲戒，除依照本辦法所列標準訂定外，得酌量下列各款\n之情形為加重或減輕： \n一、行為之動機、目的。 \n二、行為時所受之刺激。 \n三、行為之手段。 \n四、行為人之生活狀況、品行。 \n五、行為人與被害人平日之關係。\n六、行為所生之危險或損害程度。\n七、行為後之態度。 \n第十三條  學生於學校處理其個人懲戒處分之過程中，湮滅證據、故意提供或唆\n使他人提供不實證據或資料者，得加重處分。 \n第十四條  學校為發現懲戒事實之真相，應委請被付懲戒學生之導師協助進行瞭\n解，並請其以書面告知。 \n第十五條  學校審議懲戒案件應通知當事學生到會陳述，並應請相關導師列席會\n議。\n第十六條  \n一、學生個人之行為接受懲戒，應由學校以書面通知本人。學生個人\n之行為記小過以上者，應由學校以書面通知家長或監護人。 \n二、學生個人之行為接受懲戒，其行為致使特定他人權益受損害者，\n經學校認為有其必要，亦得以書面轉知該特定被害人。 \n \n第十七條  前條獎懲之通知，應附記下列事項： \n           一、學生初犯本辦法之規定而未達記大過之處分者，得依本校學生消\n過實施要點，申請以愛校服務代替之。 \n           二、學生對獎懲如有不服時，得依本校學生申訴評議辦法所訂之期限\n提出申訴。 \n第十八條  學生於在學期間有勒令退學或開除學籍之懲戒事由，且於畢業或退學'
│   │   ),
│   │   Document(
│   │   │   id='126c1097-6f34-4250-a7f5-d6f8bf30c74d',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   │   'total_pages': 6,
│   │   │   │   'page': 5,
│   │   │   │   'page_label': '6'
│   │   │   },
│   │   │   page_content='6  \n後一年內被提報懲戒者，學生獎懲委員會得懲戒之。 \n          前項情形，應通知受懲戒人到場陳述意見。受懲戒人經合法通知且無\n正當理由不到場者，學生獎懲委員會得不待其陳述作成處分。受懲戒\n人未經合法通知者，應停止懲戒程序。 \n第十九條  本辦法經學生輔導委員會議通過，送校務會議核備報請教育部備查後 \n  ，自發布日施行。'
│   │   ),
│   │   Document(
│   │   │   id='c4a1d378-c1a8-4e83-8c21-e3f386af6b9b',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-04T09:49:06+08:00',
│   │   │   │   'title': 'Microsoft Word - 04-009',
│   │   │   │   'source': '/content/學生個人獎懲辦法.pdf',
│   │   │   │   'total_pages': 6,
│   │   │   │   'page': 3,
│   │   │   │   'page_label': '4'
│   │   │   },
│   │   │   page_content='4  \n五、於校園內不法儲存危險物或違禁物。 \n六、不法持有毒品、安非他命或其他麻醉藥品。 \n七、不法侵入學校廳舍、他人研究室、或寢室，情節嚴重。 \n八、私自頂讓床位並獲取不當利益、霸佔學校宿舍床位、或排斥他人合\n法住進。 \n九、惡意侵入電腦網站破壞系統、資料或發送郵件炸彈危及電腦主機安\n全、干擾他人電磁紀錄之處理。 \n十、利用電腦網路或以其他方式販售、提供、或教唆製造不法商品。 \n十一、管理公物未盡善良管理人之注意義務，致生毀損、滅失、短少，\n或管理公款有浮報、挪用、帳目不清之情事，情節嚴重。 \n十二、毆人或與人互毆，情節嚴重。 \n十三、侵犯他人智慧財產權，情節嚴重。 \n十四、其他觸犯刑事法律之行為，經法院有罪判決確定或學校查證屬實 \n，情節輕微。 \n十五、污染校產，違反環安衛相關法令規定，情節重大者。\n十六、違反學術倫理，情節嚴重。 \n十七、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員\n會查證屬情節嚴重，並移送懲處。 \n十八、性侵害，經本校性別平等教育委員會或性騷擾防治委員會查證\n屬實並移送懲處。 \n十九、親密關係暴力行為，情節嚴重，且經法院核發有保護令或有具\n體事證者。 \n第十條 學生個人有下列各款情形之一者，予以勒令退學： \n一、毆打師長。 \n二、考試作弊，情節嚴重。 \n三、違反學術倫理，情節極為嚴重。\n四、蓄意傷人，情節嚴重。 \n五、不法販賣或製造毒品、安非他命或其他麻醉藥品。 \n六、其他觸犯刑事法律之行為，經法院有罪判決確定或學校查證屬實。 \n七、積滿三次大過。 \n八、性騷擾、性霸凌，經本校性別平等教育委員會或性騷擾防治委員 \n會查證屬情節極為嚴重，並移送懲處。 \n九、性侵害，經本校性別平等教育委員會或性騷擾防治委員會查證屬 \n情節嚴重，並移送懲處。'
│   │   ),
│   │   Document(
│   │   │   id='46697331-ad3a-45a7-b2a1-a100b169eb14',
│   │   │   metadata={
│   │   │   │   'producer': 'Microsoft® Word 2016',
│   │   │   │   'creator': 'Microsoft® Word 2016',
│   │   │   │   'creationdate': '2021-04-25T21:01:28+08:00',
│   │   │   │   'author': 'joyceguo',
│   │   │   │   'moddate': '2021-05-

KeyboardInterrupt: Interrupted by user

## 範例: RAG: 多檔案 Retrieval

In [9]:
# 1. 載入多 PDF 文件
from glob import glob
paths = glob("*.pdf")
documents = []
for path in paths:
    loader = PyPDFLoader(path)
    documents += loader.load()

pprint(documents)

[
│   Document(
│   │   metadata={
│   │   │   'producer': 'Microsoft® Word 2010',
│   │   │   'creator': 'Microsoft® Word 2010',
│   │   │   'creationdate': '2017-03-16T11:37:36+08:00',
│   │   │   'author': 'Administrator',
│   │   │   'moddate': '2017-03-16T11:37:36+08:00',
│   │   │   'source': '消過實施要點.pdf',
│   │   │   'total_pages': 2,
│   │   │   'page': 0,
│   │   │   'page_label': '1'
│   │   },
│   │   page_content='1  \n國立臺灣大學學生消過實施要點 \n88.6.4 本校第8 次學生輔導委員會通過 \n99.6.3 本校第28 次學生輔導委員會修正通過 \n101.6.21 本校第32 次學生輔導委員會修正通過 \n106.1.17 本校第41 次學生輔導委員會通過\n第一條 為發揮教育輔導功能，激勵學生改過向善，依「國立臺灣大學學生個\n人獎懲辦法」第三條第四項訂定「國立臺灣大學學生消過實施要點」\n(以下簡稱本要點)。 \n第二條 本校學生初次觸犯「國立臺灣大學學生個人獎懲辦法」之規定而未達\n記大過之處分且具悔意者，得申請消過自新。初犯者未申請消過自新，\n再犯時仍得申請，但以一次為限。 \n第三條 受懲學生得於懲戒處分或申訴決定確定之次日起十日內，經導師同意\n後，向學生事務處生活輔導組 (以下簡稱生活輔導組 )提出申請，若期\n限之末日為本校之休假日，以休假日末日之次日代之；因不可抗力致\n逾期未提出申請者，得於不可抗力事由終了後十日內提出申請。 \n第四條 受懲學生申請消過自新並完成愛校服務者，經生活輔導組核准申請後，\n該懲戒處分暫停公告。 \n第五條 學生消過執行原則如下： \n一、受懲學生應完成規定之愛校服務時數，始得提出消過審議案。 \n二、愛校服務時數：書面告誡者，從事愛校服務四小時；記申誡一次\n者，從事愛校服務十小時；記小過一次者，從事愛校服務三十小\n時；餘類推。以上愛校服務應分日實施，每日以不超過四小時為\n原則，如有超過四小時之特殊必要，須事先獲得導師及生活輔導\n組同意。 \n三、愛校服務工作安排：原則上由導師協同生活輔導組指定具公益性\n之適當工作。愛校服務應於本校校區環境執行之。 \n四、愛校服務應於指定工作後二個月內執行完畢。如有特殊情況者，\n由學生獎懲委員會另訂執行期限。 \n五、愛校服務審查：由導師審查愛校服務之成效，未通過者得依本條\n第三款再指定工作，並以一次為限。導師所填寫之審查意見，宜\n說明學生對於受懲戒原因之悔意，及明確表達是否推薦學生消過\n之原因。 \n【106.2.18發布】'
│   ),
│   Document(
│   │   metadata={
│   │   │   'producer': 'Microsoft® Word 2010',
│   │   │   'creator': 'Microsoft® Word 2010',
│   │   │   'creationdate': '2017-03-16T11:37:36+08:00',
│   │   │   'author': 'Administrator',
│   │   │   'moddate': '2017-03-16T11:37:36+08:00',
│   │   │   'source': '消過實施要點.pdf',
│   │   │   'total_pages': 2,
│   │   │   'page': 1,
│   │   │   'page_label': '2'
│   │   },
│   │   page_content='2  \n六、受懲學生應接受心理輔導者，由學生事務處心理輔導中心填具評\n估表。 \n第六條 受懲學生於愛校服務執行完成後，由其導師將「愛校服務審查意見\n表」、「心理輔導評估表」、「愛校服務紀錄表」送交生活輔導組，\n提經學生獎懲委員會審定可否消過，同意者取消其懲戒處分；不同意\n者維持其懲戒處分，並予以公告。 \n獎懲委員會審定時，得通知受懲學生列席說明。 \n第七條 受懲學生於取消懲戒處分後一年內，再受申誡以上之懲戒處分，恢復\n原懲戒處分，並予以公告。 \n第八條 本要點經學生輔導委員會議通過後發布實施。'
│   ),
│   Document(
│   │   metadata={
│   │   │   'producer': '',
│   │   │   'creator': '',
│   │   │   'creationdate': '2024-07-29T14:30:49-07:00',
│   │   │   'title': '',
│   │   │   'author': 'weiwei',
│   │   │   'subject': '',
│   │   │   'keywords': '',
│   │   │   'moddate': '2024-07-29T14:30:49-07:00',
│   │   │   'source': '台大行事曆.pdf',
│   │   │   'total_pages': 2,
│   │   │   'page': 0,
│   │   │   'page_label': '1'
│   │   },
│   │   page_content='週 辦理事項\n次 日 一 二 三 四 五 六\n\xa01日 113學年度第一學期開始\n\xa01日 公告113學年度第一學期全校課程\n\xa01日 第一學期學生（不含分發入學錄取之大一新生）繳交學雜費開始（繳交至8月30日止；碩士班及博士班\n新生繳交至8月14日止）\n\xa05日 學士班新生、轉學生及碩、博士班生網路申請抵免學分開始(至8月13日中午12時截止)\n\xa08日 學士班新生(含僑生、國際學位生)網路線上註冊開始(至8月30日止)\n\xa08日 碩、博士班新生(國際學位生)網路線上註冊開始(至8月30日止)\n\xa08日 碩、博士班新生(含僑生、陸生)網路線上註冊開始(至8月15日止)\n12日 學士班轉學生網路線上註冊開始（至8月22日止）\n12日 僑生、國際學生華語文及英文能力測驗\n12日 第二學期碩、博士班學生論文繳交截止\n19日 分發入學錄取之大一新生繳交學雜費開始（繳交至8月30日止）\n19日 學士班分科測驗管道入學新生網路申請抵免學分開始(至8月22日中午12時截止)\n19日 初選第一階開始(至8月21日截止)\n19日 新進教師研習營(至8月21日止)\n22日 113年度基礎學科免修認證考試(至8月24日)\n26日 學士班新生健檢(至8月30日止，健檢辦理院系請參照新生入學服務網)\n26日 初選第二階開始(至8月27日截止)\n26日 國際學生迎新週(至9月1日止)\n27日 新生學習入門書院(含新生音樂祭及社團市集)(至8月30日止)\n30日 教師補交112學年第二學期成績截止\n31日 國際學生華語課程分班測驗\n\xa01日 新生開學典禮\n\xa02日 113學年度第一學期上課開始\n\xa02日 網路加退選課程開始(至9月14日下午6時截止)\n\xa02日 補考開始(至9月3日止，限已核准期末考試請假者參加補考)\n\xa02日 碩、博士班生學位考試申請開始(至12月2日截止)\n\xa06日 大三學生擬參加114學年度碩士班甄試排名提前畢業申請截止\n\xa07日 社團聯展\n2 8 9 10 11 12 13 14 14日 網路加選課程截止(下午6時截止)\n15日 網路退選課程截止(系統開放至9月16日上午8時)\n17日 中秋節(放假日)\n18日 停修申請開始(至11月22日截止)\n23日 第一學期非學系申請服務學習開課(至10月18日截止)\n25日 網路確認選課結果開始(至9月27日截止)\n28日 教師節\n29 30 30日 第一次進階英語網路申請免修開始(至10月4日截止，日期暫定以公告為準)\n1 2 3 4 5\n10日 國慶日(放假日)\n11日 休、退學學生退2/3學雜費截止\n14日 第二階段學分費等各項費用繳費開始(至10月25日止)\n14日 期中教學意見調查開始(至10月25日止)\n18日 第一學期第一次教務會議\n19日 第一學期第一次校務會議\n19日 全校運動會游泳賽\n8 20 21 22 23 24 25 26 21日 期中考試開始(至10月25日止)，若教師另有訂定其他日期者，從其規定。\n31日 學士班第一學期應屆畢業生自行修讀輔系並取得輔系資格畢業申請截止\n31日 學士班第一學期應屆畢業生符合跨域專長資格畢業申請截止\n1 2\n10 3 4 5 6 7 8 9  9日 校慶園遊會\n15日 本校校慶( 停課不停班)\n15日 休、退學學生退1/3學雜費截止\n22日 停修申請截止\n22日 全校運動會(至11月23日止， 停課不停班)\n13 24 25 26 

In [10]:
# 印出Document內容
for doc in documents:
    print(doc.page_content)
    print("=== \n")

1  
國立臺灣大學學生消過實施要點 
88.6.4 本校第8 次學生輔導委員會通過 
99.6.3 本校第28 次學生輔導委員會修正通過 
101.6.21 本校第32 次學生輔導委員會修正通過 
106.1.17 本校第41 次學生輔導委員會通過
第一條 為發揮教育輔導功能，激勵學生改過向善，依「國立臺灣大學學生個
人獎懲辦法」第三條第四項訂定「國立臺灣大學學生消過實施要點」
(以下簡稱本要點)。 
第二條 本校學生初次觸犯「國立臺灣大學學生個人獎懲辦法」之規定而未達
記大過之處分且具悔意者，得申請消過自新。初犯者未申請消過自新，
再犯時仍得申請，但以一次為限。 
第三條 受懲學生得於懲戒處分或申訴決定確定之次日起十日內，經導師同意
後，向學生事務處生活輔導組 (以下簡稱生活輔導組 )提出申請，若期
限之末日為本校之休假日，以休假日末日之次日代之；因不可抗力致
逾期未提出申請者，得於不可抗力事由終了後十日內提出申請。 
第四條 受懲學生申請消過自新並完成愛校服務者，經生活輔導組核准申請後，
該懲戒處分暫停公告。 
第五條 學生消過執行原則如下： 
一、受懲學生應完成規定之愛校服務時數，始得提出消過審議案。 
二、愛校服務時數：書面告誡者，從事愛校服務四小時；記申誡一次
者，從事愛校服務十小時；記小過一次者，從事愛校服務三十小
時；餘類推。以上愛校服務應分日實施，每日以不超過四小時為
原則，如有超過四小時之特殊必要，須事先獲得導師及生活輔導
組同意。 
三、愛校服務工作安排：原則上由導師協同生活輔導組指定具公益性
之適當工作。愛校服務應於本校校區環境執行之。 
四、愛校服務應於指定工作後二個月內執行完畢。如有特殊情況者，
由學生獎懲委員會另訂執行期限。 
五、愛校服務審查：由導師審查愛校服務之成效，未通過者得依本條
第三款再指定工作，並以一次為限。導師所填寫之審查意見，宜
說明學生對於受懲戒原因之悔意，及明確表達是否推薦學生消過
之原因。 
【106.2.18發布】
=== 

2  
六、受懲學生應接受心理輔導者，由學生事務處心理輔導中心填具評
估表。 
第六條 受懲學生於愛校服務執行完成後，由其導師將「愛校服務審查意見
表」、「心理輔導評估表」、「愛校服務紀錄表」送交生活輔導組，
提經學生獎懲委員會審定可否消過，同意者取消其懲戒處分；不同意
者維持其懲戒處分，

In [11]:
# 3. 建立向量資料庫（知識庫）
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
db = FAISS.from_documents(documents, embedding)

# 4. 用 RetrievalQA 包一層問答機制
retriever = db.as_retriever()
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True  # 回傳來源
)

# 5. 開始問問題
while True:
    query = input("請問你想問什麼？（輸入 'exit' 結束）\n")
    if query.lower() == "exit":
        break
    response = qa_chain.invoke(query)
    pprint(response["answer"])

請問你想問什麼？（輸入 'exit' 結束）



'The provided documents do not contain information about what the president said about Michael Jackson.\n'

請問你想問什麼？（輸入 'exit' 結束）



'The provided documents do not contain information about what the president said about Michael Jackson.\n'

請問你想問什麼？（輸入 'exit' 結束）
exit


# 客製化Prompt

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

# 定義系統提示，告訴模型如何使用檢索到的內容
system_prompt = """你是一個問答任務的助手。
請使用以下檢索到的內容來回答問題。
如果你不知道答案，就直接說不知道。
參考內容: {context}:"""

# 定義一個問題
question = """扶老太太過馬路會有什麼獎勵?"""

# 檢索與問題相關的文件
docs = retriever.invoke(question)

# 將檢索到的文件內容合併成一個字串
docs_text = "".join(d.page_content for d in docs)

# 用檢索到的內容填充系統提示
system_prompt_fmt = system_prompt.format(context=docs_text)

# 生成回應
response = model.invoke([
    SystemMessage(content=system_prompt_fmt),
    HumanMessage(content=question)
])

In [ ]:
pprint(response)

#### Chatbot

In [15]:
system_prompt = """
你是一個問答任務的助手。
請使用以下檢索到的內容來回答問題。
如果你不知道答案，就直接說不知道。
參考內容: {context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])
# 使用進階retriever
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
retriever = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever( # 基本retriever是db預設的
        search_kwargs={"k": 10}
    ),
    llm=model # 用哪個 LLM 來產生多樣化的query文字
)

# 建立 RetrievalQA Chain，並注入自訂 Prompt
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},  # 把自訂 prompt 注入 chain
    return_source_documents=True
)

# 問答流程
while True:
    query = input("請問你想問什麼？（輸入 'exit' 結束）\nYou: ")
    if query.lower() == "exit":
        break
    response = qa_chain.invoke({"query": query})
    print("💬 回答：", response["result"])
    print("📚 來源：", [d.metadata.get("source", "unknown") for d in response["source_documents"]])
    print("-" * 50)

請問你想問什麼？（輸入 'exit' 結束）
You: 在學校開賭場的問題
💬 回答： 根據您提供的資料，在學校開賭場的行為屬於「有公然賭博之行為」，這會被記小過。

參考來源：第3條 學生個人之懲處方式，其中「小過」的定義包含「有公然賭博之行為」。
📚 來源： ['學生個人獎懲辦法.pdf', '學生個人獎懲辦法.pdf', '學生個人獎懲辦法.pdf', '學生個人獎懲辦法.pdf', '學生個人獎懲辦法.pdf', '消過實施要點.pdf', '消過實施要點.pdf', '台大行事曆.pdf', '學生個人獎懲辦法.pdf', '台大行事曆.pdf']
--------------------------------------------------
請問你想問什麼？（輸入 'exit' 結束）
You: 奧運得到金牌的獎勵?
💬 回答： 根據檢索到的「國立臺灣大學學生個人獎懲辦法」，並沒有明確提到奧運得到金牌的獎勵。

不過，辦法中提到以下幾項獎勵可能會適用：

*   **記大功**：
    *   「參加國際性比賽表現特優。」
    *   「有其他相當之情事。」

奧運獲得金牌無疑是國際性比賽的最高表現，因此很有可能符合「參加國際性比賽表現特優」的條件，並給予記大功的獎勵。
📚 來源： ['學生個人獎懲辦法.pdf', '學生個人獎懲辦法.pdf', '消過實施要點.pdf', '消過實施要點.pdf', '學生個人獎懲辦法.pdf', '學生個人獎懲辦法.pdf', '台大行事曆.pdf', '台大行事曆.pdf', '學生個人獎懲辦法.pdf', '學生個人獎懲辦法.pdf']
--------------------------------------------------


KeyboardInterrupt: Interrupted by user